In [57]:
import numpy as np
import msicpe.san as san
from plotly import express as px
import scipy.signal as signal


dat = np.load('data_partie_2/etude/signal_10.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
s = dat['s'] # extraction du signal utile
t = dat['t'] # extraction du vecteur temps associé au signal

sf = san.trans_fourier(s,t)
nu = sf[0]
spc = np.abs(sf[1])

px.line(x = t, y = s, title= "Signal temporel n°10", labels={"x": "Temps (s)", "y": "Amplitude"}).show()
px.line(x = nu, y = spc, range_x = [0,1500], title= "Spectre du signal n°10", labels={"x": "Fréquence (Hz)", "y": "Amplitude"}).show()
px.line(x = nu, y = np.log10(spc), range_x = [0,1500],title="Spectre du signal n°10 en log10", labels={"x": "Fréquence (Hz)", "y": "Amplitude log10"})


In [58]:
H=15 #Apres test, le minimum qui fonctionne est H=12, mais pour plus de marge, on prend H = 15


Vvec_moy = [[] for i in range(5)]
lettres = ['a', 'e', 'i', 'o', 'u']
a = range(5)

for lettre,j in zip(lettres, a):
    signaux = []
    for i in range(5):
        dat = np.load(f'data_partie_2/apprentissage/{lettre}_{i+1}.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
        s = dat['s'] # extraction du signal utile
        t = dat['t'] # extraction du vecteur temps associé au signal

        nu, S = san.trans_fourier(s, t)
        S = np.abs(S)

        s_centre= s - np.mean(s)
        nu, S_norm = san.trans_fourier(s_centre, t)
        S_norm = np.abs(S_norm)
        S_norm /= np.max(S_norm)

        f0 = san.detect_fondamentale(S_norm, nu)

        freq_list = [f0 * k for k in range(1, H+1)]

        freq_peaks, amp_peaks = san.detect_pics(S_norm, freq_list, nu)

        vec = np.zeros(H+1)
        vec[0] = np.log10(np.max(S_norm))
        vec[1:] = np.log10(amp_peaks)
        signaux.append(vec)


    k = np.arange(H+1)
    vec_moy = np.mean(signaux, axis=0)
    Vvec_moy[j] = vec_moy

    k = list(range(H+1))
    indice = []
    ampl = []
    labels = []

    for i in range(5):
        indice.extend(k)
        ampl.extend(signaux[i])
        labels.extend([f"Son {i+1}"] * (H+1))

    indice.extend(k)
    ampl.extend(vec_moy)
    labels.extend(["Moyenne"] * (H+1))

    fig = px.line(x=indice, y=ampl, color=labels, markers=True,title=f"Voyelle: {lettre} - Spectre (log10) du vecteur moyen", labels={"x": "Indice d'harmonique k", "y": "Amplitude log10"}).show()



In [59]:
def norme(s1,s2):
        return np.sqrt(sum((s1-s2)**2))

for lettre in lettres:
    for j in [6,7]:
        
        dat = np.load(f'data_partie_2/test/{lettre}_{j}.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
        s = dat['s'] # extraction du signal utile
        t = dat['t'] # extraction du vecteur temps associé au signal

        s_centre= s - np.mean(s)
        nu, S_norm = san.trans_fourier(s_centre, t)
        S_norm = np.abs(S_norm)
        S_norm /= np.max(S_norm)

        f0 = san.detect_fondamentale(S_norm, nu)

        freq_list = [f0 * k for k in range(1, H+1)]

        freq_peaks, amp_peaks = san.detect_pics(S_norm, freq_list, nu)

        vec = np.zeros(H+1)
        vec[0] = np.log10(np.max(S_norm))
        vec[1:] = np.log10(amp_peaks)



        L_norme = []
            
        for i in range(len(lettres)):
            L_norme.append(norme(vec, Vvec_moy[i]))

        if lettres[L_norme.index(min(L_norme))] != lettre:
            print(f"ce n'est pas la lettre {lettre}_{j}. La lettre trouvée est {lettres[L_norme.index(min(L_norme))]}")




In [62]:


dat = np.load(f'data_partie_2/etude/signal_10.npz') # dat est un dictionnaire contenant les données contenues dans le fichier .npz
s = dat['s'] # extraction du signal utile
t = dat['t'] # extraction du vecteur temps associé au signal

s_centre= s - np.mean(s)
nu, S_norm = san.trans_fourier(s_centre, t)
S_norm = np.abs(S_norm)
S_norm /= np.max(S_norm)

f0 = san.detect_fondamentale(S_norm, nu)

freq_list = [f0 * k for k in range(1, H+1)]

freq_peaks, amp_peaks = san.detect_pics(S_norm, freq_list, nu)

vec = np.zeros(H+1)
vec[0] = np.log10(np.max(S_norm))
vec[1:] = np.log10(amp_peaks)

L_norme = []

for i in range(len(lettres)):
    L_norme.append(norme(vec, Vvec_moy[i]))

voyelle = lettres[L_norme.index(min(L_norme))]
print(f"La voyelle trouvée est {voyelle}")

ind = []
a = []
label = []

for i in range (2):
    ind.extend(k)

a.extend(Vvec_moy[0])
a.extend(vec)

label.extend(["Moyenne a"] * (H+1)) 
label.extend(["Signal n°10"] * (H+1))

fig = px.line(x=ind, y=a, color=label, markers=True,title=f"Spectre (log10)", labels={"x": "Indice d'harmonique k", "y": "Amplitude log10"}).show()


La voyelle trouvée est a
